In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd 
import nilearn
from nilearn import plotting
from fragmenter import Fragment
from fragmenter import adjacency
from fragmenter import RegionExtractor

This is a simple demo of the parcellation fragmenter, from simple use to some benchmarks. The first step is to load a template surface (in this case, the inflated fsaverage). Use your own path_to_data to load the template surface.

In [ ]:
testSurface = nib.freesurfer.read_geometry(
   r'C:\Users\Amanda\Desktop\parcellation_fragmenter\data\freesurfer\fsaverage\surf\rh.inflated'
)

Surfaces are a 2D triangular mesh, so that each data-containing vertex is tied to 2 neighbors by a triangle. When loading it, you get a tuple of two arrays: the coordinates of each vertex and the vertex's direct neighbors (faces).

In [ ]:
testSurface

The fsaverage mesh is composed of ~164k vertices, which we can confirm here.

In [ ]:
np.shape(testSurface[0])[0]

In order to parcellate a surface, we derive the neighbors of each vertex. You can consider this to be an adjacency matrix that allows for standard clustering methods to be computed on it. The `adjacency` function will give you this list (in this example, we will just look at the neighbors of vertex 2).

In [ ]:
# Create a surface adjacency object
M = adjacency.SurfaceAdjacency(testSurface[0], testSurface[1])

# Generate adjacency list
M.generate()

# Visualize
M.adj[1]

In order to fragment a surface, we first create a `Fragment` object that takes in the desired number of clusters, and whether you want to use pretty colors (True/False). In this case, we will run a simple 10 parcel example.

In [ ]:
testFragment = Fragment.Fragment(n_clusters=10)

Once the object is defined, you can check that the number of clusters is right.

In [ ]:
testFragment.n_clusters

Now we are ready to fit the number of clusters onto the provided surface. Coordinates and vertices are provided separately (meaning that you could just cluster custom, non-cortical meshes). The following example uses mini batch k-means clustering for parcellating our surface. Other clustering algorithms available are Gaussian Mixture Models ('gmm') and Agglomerative Clustering and Ward ('ward').

In [ ]:
testFragment.fit(vertices = testSurface[0], faces=testSurface[1], method = 'k_means')

This will add an attribute to `testFragment` called `label_`, which is the parcel label associated with each vertex. We can confirm this by checking that length of the resulting vector is equal to the number of vertices in fsaverage.

In [ ]:
np.size(testFragment.label_)

We can also see that the number of labels produced is equal to our desired number of clusters (10).

In [ ]:
np.unique(testFragment.label_)

And now we can plot the 10 parcels onto a surface (**FIX THIS SO WE CAN USE VIEW_SURF INSTEAD**).

In [ ]:
plotting.plot_surf_roi(list(testSurface), testFragment.label_);

**TO ADD**
- ROI-specific parcellation
- Null model explanation (once Kristian finishes implementing the class)

If we want to parcellate based on regions, we can pass the label file (.annot) to map all the vertices of this ROI. Region Extractor generates a dictionary with the FreeSurfer parcellation areas. It tells us which vertices correspond to each brain area. 

In [ ]:
label_file = r'C:\Users\Amanda\Desktop\parcellation_fragmenter\data\freesurfer\fsaverage\label\lh.aparc.annot'
ext = RegionExtractor.Extractor()
label_table = ext.get_label_table(label_file)
#label_table

To parcellate a given brain region, call it from the dictionary created by RegionExtractor using label_table. This returns the vertices associated with the region.

In [ ]:
label_table['cuneus']

In [ ]:
testRegion = Fragment.Fragment(n_clusters=10)
testRegion.fit(vertices = testSurface[0], faces=testSurface[1], method = 'k_means', parcels = label_table, rois= ['entorhinal'])

In [ ]:
testRegion.label_.shape

In [ ]:
plotting.plot_surf_roi(list(testSurface), testRegion.label_, cmap='Spectral');

In [ ]:
testSurface[1].shape

In [ ]:
np.unique(testRegion.label_)